In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import xlrd
import pathlib
from pathlib import Path
import re

### Skapa en Data Frame utifrån excelfiler i en Mapp + ta bort duplikater

In [5]:
### vilken mapp vill du kolla i?
mapp = 'protein-peptide_sample_inf'
## Skapar en lista över excelfilerna i mappen
files = os.listdir(mapp)
files_xls = [f for f in files if f[-3:] == 'lsx']
files_xls
## Läser in filerna 1 och 1 och appendar dom i df
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(mapp + '/' + f)
    ## Tar bort duplikat peptider eftersom protein-peptid filerna har duplicater inom t.ex hemoglobin
    data.drop_duplicates(subset='Peptide', inplace=True, ignore_index=True)
    df = df.append(data)

### Formaterar Data Framen utifrån vilken fil den skapats utifrån

In [185]:
## Vad för typ av fil hanterar du? (vilka kolummner önskas ha kvar)
peptide = ['Accession', 'Peptide', 'Mass', 'Length', '#Feature', '-10lgP']
protein = ['Protein Accession', 'Peptide', 'Mass', 'Length', '#Feature', '-10lgP', 'Start', 'End']
## Formatera dataframen ändra [] för att byta formatering
df = df[protein]
df

,Protein Accession,Peptide,Mass,Length,#Feature,-10lgP,Start,End
0,sp|P69905|HBA_HUMAN,K.TNVKAAWGKVGAHAGEYGAEALERMFL.S,2875.4438,27,4,92.76,9,35
1,sp|P69905|HBA_HUMAN,K.TNVKAAWGKVGAHAGEYGAEALER.M,2484.2510,24,15,91.89,9,32
2,sp|P69905|HBA_HUMAN,K.AAWGKVGAHAGEYGAEALERMFL.S,2433.1899,23,4,91.79,13,35
3,sp|P69905|HBA_HUMAN,V.DDMPNALSALSDLHAHKLRVDPV.N,2513.2695,23,4,91.73,75,97
4,sp|P69905|HBA_HUMAN,M.VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFL.S,3585.8401,34,9,90.15,2,35
...,...,...,...,...,...,...,...,...
4742,sp|P08603|CFAH_HUMAN,V.TAPENGKIV.S,927.5025,9,1,23.46,150,158
4743,sp|P06703|S10A6_HUMAN,K.LQDAEIAR.L,914.4821,8,2,25.16,48,55
4744,sp|P06703|S10A6_HUMAN,A.LIYNEALKG,1019.5651,9,1,21.63,82,90
4745,sp|Q7KZ85|SPT6H_HUMAN,S.HQVQEWNRQRTMAIER.A,2081.0337,16,0,24.38,700,715


### Tar bort modifikations information från Peptidkolumnen + skapa Count kolummn + Feature cutoff, OBS, finns nu alltså duplikater igen!!

In [196]:
## Tar bort allt ur kolummn som inte är bokstäver
df['Peptide'] = df['Peptide'].str.replace('[^a-zA-Z]', '')
## sortera bort om feature<4
df = df.loc[(df['#Feature'] >= 4)] 
## Skapa column "count"
df.insert(5,'count', 1)
df

<ipython-input-196-1075f3eb7dfc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Peptide'] = df['Peptide'].str.replace('[^a-zA-Z]', '')


,Protein Accession,Peptide,Mass,Length,#Feature,-10lgP,Start,End
0,sp|P69905|HBA_HUMAN,KTNVKAAWGKVGAHAGEYGAEALERMFLS,2875.4438,27,4,92.76,9,35
1,sp|P69905|HBA_HUMAN,KTNVKAAWGKVGAHAGEYGAEALERM,2484.2510,24,15,91.89,9,32
2,sp|P69905|HBA_HUMAN,KAAWGKVGAHAGEYGAEALERMFLS,2433.1899,23,4,91.79,13,35
3,sp|P69905|HBA_HUMAN,VDDMPNALSALSDLHAHKLRVDPVN,2513.2695,23,4,91.73,75,97
4,sp|P69905|HBA_HUMAN,MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLS,3585.8401,34,9,90.15,2,35
...,...,...,...,...,...,...,...,...
4742,sp|P08603|CFAH_HUMAN,VTAPENGKIVS,927.5025,9,1,23.46,150,158
4743,sp|P06703|S10A6_HUMAN,KLQDAEIARL,914.4821,8,2,25.16,48,55
4744,sp|P06703|S10A6_HUMAN,ALIYNEALKG,1019.5651,9,1,21.63,82,90
4745,sp|Q7KZ85|SPT6H_HUMAN,SHQVQEWNRQRTMAIERA,2081.0337,16,0,24.38,700,715


### Skapa en ny Data Frame med de peptider som endast finns i alla (df_same)

In [151]:
## Skapa en DataFrame av de som endast finns i alla
df_same = pd.DataFrame()
df_same = df

## Gruppera efter Peptide och summera siffror (mergear duplikater)
df_same = df_same.groupby(['Peptide'], as_index=False).sum()
## Sortera efter count
df_same = df_same.sort_values(by=['count'], axis=0, ascending=False)

## OBS!! måste ändra till rätt filantal! (Skapa nytt dataset med endast count 3 (denna har alltså funnits i alla filer = same))
df_same = df_same.loc[df_same['count'] == 3] 

## endast de peptider som finns i alla
df_same

,Peptide,Mass,Length,#Feature,count,-10lgP,Start,End
1062,P.HFDLSHGSAQ.V,3292.4670,30,18,3,150.33,138,165
1418,V.NVDEVGGEALGRLLV.V,4619.4771,45,26,3,175.67,60,102
889,L.SHGSAQVK.G,2437.2423,24,15,3,124.66,150,171
244,E.VGGEALGRLLV.V,3247.9344,33,14,3,132.02,72,102
1438,V.SLGSPSGEVSHPRKT.R,4613.3544,45,28,3,138.60,975,1017
...,...,...,...,...,...,...,...,...
986,M.VHLTPEEKSAV.T,3625.9200,33,78,3,161.89,6,36
77,A.HLDNLKGT.F,2689.4148,24,13,3,113.67,234,255
989,M.VHLTPEEKSAVTAL.W,4481.4270,42,35,3,163.38,6,45
80,A.HLDNLKGTFA.T,3343.7310,30,23,3,140.38,234,261


### Skapar kolummner för N-cut och C-cut + delar Mass, Length, RT på count

In [ ]:
df['N-cut']=df['Peptide'].apply(lambda x: x[0:4])
df['C-cut']=df['Peptide'].apply(lambda x: x[-4::1])
df['Mass']=df['Mass']/df['count']
df['Length']=df['Length']/df['count']
df['RT']=df['RT']/df['count']
df = df.sort_values(['count','#Feature'], ascending=False)

### Skapar nya Data Frames för Peptiderna utifrån cutsite

In [ ]:
#Here we sort on the C and N-cut
df_N=df.groupby(['Accession', 'N-cut']).sum()
df_N['Mass']=df_N['Mass']/df_N['count']
df_N['Length']=df_N['Length']/df_N['count']
df_N['RT']=df_N['RT']/df_N['count']
df_N=df_N.sort_values(['count'], ascending=False)

#sorted by accession and then C-cut. KGEE is found 48 times in APOC2
df_C=df.groupby(['Accession', 'C-cut']).sum()
df_C['Mass']=df_C['Mass']/df_C['count']
df_C['Length']=df_C['Length']/df_C['count']
df_C['RT']=df_C['RT']/df_C['count']
df_C=df_C.sort_values(['count'], ascending=False)

df_N.head()

### Skapar nya Data Frames för Peptiderna grupperade(df_N, df_C)

In [ ]:
df_N= df.groupby(['Accession','N-cut']).sum() #groups original DataFrame by protein and n-cut. Makes protein and cut indexes
df_N = df_N.sort_values(['count','#Feature'], ascending=False) #sorts these values by count and feature
df_N.reset_index(['Accession'], inplace=True) #removes protein from index, only cut left
df_N['N-grouped']=group(df_N.index) #creates column of N-grouped based on index, cut

df_N=df_N.groupby(['N-grouped']).sum() #groups by N-grouped instead

df_N['Mass']=df_N['Mass']/df_N['count'] #this is fucked
df_N['Length']=df_N['Length']/df_N['count'] 
df_N['RT']=df_N['RT']/df_N['count']

df_N=df_N.sort_values(['count','#Feature'], ascending=False) #sorts the grouped index on count and feature

df_C = df.groupby(['Accession','C-cut']).sum() 
df_C = df_C.sort_values(['count','#Feature'], ascending=False)
df_C.reset_index(['Accession'], inplace=True)
df_C['C-grouped']=group(df_C.index)

df_C=df_C.groupby(['C-grouped']).sum()

df_C['Mass']=df_C['Mass']/df_C['count']
df_C['Length']=df_C['Length']/df_C['count']
df_C['RT']=df_C['RT']/df_C['count']

df_C=df_C.sort_values(['count','#Feature'], ascending=False)
df_N